In [46]:
import os
import math
import time
from queue import Queue
from collections import deque

import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

In [47]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        self.adj_edges = []

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def add_adj_edge(self, edge):
        self.adj_edges.append(edge)

    def point_is_close(self, p):
        if math.isclose(self.x,p.x) and math.isclose(self.y,p.y) and math.isclose(self.z, p.z):
            return True
        return False
            
        
class Edge:
    def __init__(self, p1, p2, visits=0):
        if p1.point_is_close(p2):
            raise Exception("Edge must have two different points. Points given: {} and {}.".format(p1, p2))
        self.p1 = p1
        self.p2 = p2
        self.visits = visits
        p1.add_adj_edge(self)
        p2.add_adj_edge(self)

    def __str__(self):
        return "{} {}".format(str(self.p1), str(self.p2))

    def __eq__(self, other):
        if self.p1 == other.p1 or self.p1 == other.p2:
            if self.p2 == other.p1 or self.p2 == other.p2:
                return True
        return False


class Face():
    def __init__(self, p1, p2, p3):
        # super().__init__(self)
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

        self.e1 = Edge(p1, p2)
        self.e2 = Edge(p2, p3)
        self.e3 = Edge(p3, p1)

        self.spoly = Polygon([[p1.x,p1.y,p1.z],[p2.x,p2.y,p2.z],[p3.x,p3.y,p3.z]])
    def __str__(self):
        return "{} {} {}".format(str(self.p1), str(self.p2), str(self.p3))

    def __eq__(self, other):
        if self.p1.point_is_close(other.p1) or self.p1.point_is_close(other.p2) or self.p1.point_is_close(other.p3):
            if self.p2.point_is_close(other.p1) or self.p2.point_is_close(other.p2) or self.p2.point_is_close(other.p3):
                if self.p3.point_is_close(other.p1) or self.p3.point_is_close(other.p2) or self.p3.point_is_close(other.p3):
                    return True
        return False

In [48]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            try:
                line = line.replace('v', '')
            except:
                #                 print("Problema na linha")
                continue
            line_ = line.split()
            try:
                points.append(
                    Point(float(line_[0]), float(line_[1]), float(line_[2])))
            except:
                #                 print("Problema no ponto")
                continue
    return points
  
        
def read_faces_from_obj(objPath):
    vertices = read_vertices_from_obj(objPath)
    faces = []
    with open(objPath, 'r') as f:
        for line in f:
            try:
                line = line.replace('f', '')
            except:
                continue
            line_ = line.split()
            try:
                new_face = Face(vertices[int(line_[0])-1], vertices[int(line_[1])-1], vertices[int(line_[2])-1])
                faces.append(new_face)
            except Exception as e:
                # print(e)
                continue
    return faces


def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'obj', 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                    i1 = j
                if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                    i2 = j
                if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')

In [49]:
def cross(a, b):
    return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

def dot(a, b):
    return (a.x*b.x)+(a.y*b.y)+(a.z*b.z)

def norma(a):
    return math.sqrt(a.x*a.x + a.y*a.y + a.z*a.z)

def angle(a, b):
    normaa = norma(a)
    normab = norma(b)
    if norma(a)==0 or norma(b) == 0:
        return -999999999
    return dot(a, b) / (norma(a)*norma(b))

def sub(a, b):
    return Point(a.x-b.x, a.y-b.y, a.z-b.z)

def normal(face):
    v1 = sub(face.p2, face.p1)    
    v2 = sub(face.p3, face.p1)
    return cross(v1,v2)

def ccw(A,B,C):
    return (C.y-A.y) * (B.x-A.x) > (B.y-A.y) * (C.x-A.x)

# Return true if line segments AB and CD intersect
def intersect(A,B,C,D):
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

#  dist = Abs(a*x1+b*y1+c*z1+d) / Sqrt(a^2+b^2+c^2)
def distance_point_to_plane(p, n, f):
    a = n.x
    b = n.y
    c = n.z
    d = a*f.p1.x + b*f.p1.y + c*f.p1.z
    dist = (a*p.x+b*p.y+c*p.z+d)/math.sqrt(a**2+b**2+c**2)
    dist = abs(dist)
    return dist

In [77]:
def check_intersection_faces(point, face, faces):
    """Checks if the faces created with point intersect 
    a set of known faces.

    Args:
        point (Point): Point to verify
        face (Face): Base face
        faces (list): List of known faces
    """
    new_faces = []
    # new_faces.append(Face(face.e1.p1, point, face.e1.p2))
    # new_faces.append(Face(face.e2.p1, point, face.e2.p2))
    # new_faces.append(Face(face.e3.p1, point, face.e3.p2))
    print(point)
    f1 = Face(face.e1.p1, point, face.e1.p2)
    f2 = Face(face.e2.p1, point, face.e2.p2)
    f3 = Face(face.e3.p1, point, face.e3.p2)
    print(f1.spoly.touches(faces[0].spoly))
    print(f2.spoly.touches(faces[0].spoly))
    print(f3.spoly.touches(faces[0].spoly))
    print(f1.spoly.intersects(faces[0].spoly))
    print(f2.spoly.intersects(faces[0].spoly))
    print(f3.spoly.intersects(faces[0].spoly))
    
    print(f1)
    print(f2)
    print(f3)
    for f in faces:
        if not f.spoly.touches(f1.spoly) and f.spoly.intersects(f1.spoly):
            print(f, "intersects", f1)
            return True
        if not f.spoly.touches(f2.spoly) and f.spoly.intersects(f2.spoly):
            print(f, "intersects", f2)
            return True
        if not f.spoly.touches(f3.spoly) and f.spoly.intersects(f3.spoly):
            print(f, "intersects", f3)
            return True
    return False


In [81]:
fs = read_faces_from_obj('obj/tetra_cilindro_broken.obj')
for f in fs: print(f)
vs = read_vertices_from_obj('obj/tetra_cilindro_broken.obj')
check_intersection_faces(vs[0], fs[0], fs)

(0.866025, -1.0, -0.5) (0.866025, 1.0, -0.5) (0.866025, -1.0, 0.5)
(0.866025, 1.0, -0.5) (0.866025, 1.0, 0.5) (0.866025, -1.0, 0.5)
(0.866025, -1.0, 0.5) (0.866025, 1.0, 0.5) (0.866025, -1.0, -0.5)
(0.866025, -1.0, -0.5)


Exception: Edge must have two different points. Points given: (0.866025, -1.0, -0.5) and (0.866025, -1.0, -0.5).

In [52]:
def check_if_face_already_exists(face_candidate, faces):
    """Returns True if face_candidate is already in faces

    Args:
        face_candidate (Face): Face candidate to enter the frontier
        faces (list): List of faces already in the frontier
    """
    for face in faces:
        if face_candidate == face:
            return True
    return False

In [53]:
def avanco_de_fronteira_3d(hull, points):
    faces = hull.copy()
    faces_obtidas = hull.copy()
    max_dist = -float("inf")
    debug = 0
    while len(faces) > 0:
#         print("Before POP",len(faces))
        candidate = None
        max_dist = -float("inf")
        f = faces.pop(0)
        print("Removed face", f)
#         print("After POP",len(faces))
        normal_1 = normal(f)
        # for i in range(len(points)):
        for p in points:
            if p.point_is_close(f.p1) or p.point_is_close(f.p2) or p.point_is_close(f.p3):
                continue
            dist = distance_point_to_plane(p, normal_1, f)
            if dist > max_dist:
                if not check_intersection_faces(p, f, faces_obtidas):
                    max_dist = dist
                    candidate = p

        if candidate is not None:
            # print("Candidate is not None.")
            f1 = Face(f.e1.p1, candidate, f.e1.p2)
            f2 = Face(f.e2.p1, candidate, f.e2.p2)
            f3 = Face(f.e3.p1, candidate, f.e3.p2)
            # print("Adding faces:")
            # print(f1)
            # print(f2)
            # print(f3)

            if not check_if_face_already_exists(f1, faces) and not f1 == f:
            # if not check_if_face_already_exists(f1, faces) and not f1 == f:
                # print("Face", f1, "not in faces.")
                faces.append(f1)
            if not check_if_face_already_exists(f2, faces) and not f2 == f:
            # if not check_if_face_already_exists(f2, faces) and not f2 == f:
                # print("Face", f2, "not in faces.")
                faces.append(f2)
            if not check_if_face_already_exists(f3, faces) and not f3 == f:
            # if not check_if_face_already_exists(f3, faces) and not f3 == f:
                # print("Face", f3, "not in faces.")
                faces.append(f3)
            
            if not check_if_face_already_exists(f1, faces_obtidas):
                faces_obtidas.append(f1)
            if not check_if_face_already_exists(f2, faces_obtidas):
                faces_obtidas.append(f2)
            if not check_if_face_already_exists(f3, faces_obtidas):
                faces_obtidas.append(f3)
        
        
        print()
        
            # faces.extend([f1, f2, f3])
            # faces_obtidas.extend([f1, f2, f3])
        debug += 1
        if debug > 500: 
            print("Rechead DEBUG limit.")
            break
    return faces_obtidas


In [58]:
obj_file = 'obj/hullCilindro.obj'
faces = read_faces_from_obj(obj_file)
vertices = read_vertices_from_obj(obj_file)
res = avanco_de_fronteira_3d(faces, vertices)
print(len(res))
write_faces(res, vertices, 'obj/tetra_cilindro.obj')

Removed face (-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (-0.866025, -1.0, -0.5) (0.866025, -1.0, -0.5) (0.0, 1.0, -1.0)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (-0.866025, -1.0, -0.5) (0.866025, 1.0, -0.5) (0.0, 1.0, -1.0)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (-0.866025, -1.0, -0.5) (0.866025, -1.0, 0.5) (0.0, 1.0, -1.0)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (-0.866025, -1.0, -0.5) (0.866025, 1.0, 0.5) (0.0, 1.0, -1.0)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (-0.866025, -1.0, -0.5) (-0.0, -1.0, 1.0) (0.0, 1.0, -1.0)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (0.0, -1.0, -1.0) (-0.0, 1.0, 1.0) (-0.866025, -1.0, -0.5)
(-0.866025, -1.0, -0.5) (0.0, 1.0, -1.0) (0.0, -1.0, -1.0) intersects (0.0, 1.0, -1.0) (-0.866025, -1.0, 0.5) (0.0, -1.0, -1.0)
(-0.866025

In [55]:
a = [f]
# print(f)
g = Face(Point(-3,0,0), Point(0,0,-4), Point(0,-3,0))
if not check_if_face_already_exists(g, a):
    print("lala")

lala
